<a href="https://colab.research.google.com/github/sgirabin/nyp-iti102-assignment/blob/main/%5BITI102%5DPart_2_4466624P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trend Analysis for Grocery Store Key&U


| Name | Student ID | Input File |
|------|------------|------------|
| Isak Rabin | 4466624P |  4466624P_sales.csv |


## Background

Grocery store Key&U is a mini-mart that has been open since 2000. The store owner has recently digitised its data. The store owner wishes to use the data collected from the past years to have a deeper insight into the business and, hopefully, will be able to use machine learning to perform predictions.

During the interview with the store owner, he wanted to be able to
* View any common trends for each month throughout the year
* View any common trends for each year


## Objective

To analyze historical sales transaction data to identify common trends across months and years.

This will help the store owner plan promotions, staffing, and inventory.

## Implementation

### Pre-requisite

Download the input file from google drive
https://drive.google.com/file/d/1dvrvIcmpXrNk7ah_U6Y4tt78HCI_zGTm/view?usp=drive_link

In [89]:
# download data.csv from google drive
!wget "https://drive.usercontent.google.com/download?id=1DXadGavnfs56AGB8cd25h4luq1NcqIcG&export=download&authuser=0&confirm=t&uuid=eef3517c-29e4-417a-b701-ba5a581b803d&at=ALoNOgkW2QjovD5DOm23UZ44vzwi:1749349835971" -O 4466624P_sales.csv

# check file content to confirm file downloaded successfully
!head  4466624P_sales.csv

--2025-06-08 03:29:58--  https://drive.usercontent.google.com/download?id=1DXadGavnfs56AGB8cd25h4luq1NcqIcG&export=download&authuser=0&confirm=t&uuid=eef3517c-29e4-417a-b701-ba5a581b803d&at=ALoNOgkW2QjovD5DOm23UZ44vzwi:1749349835971
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.215.132, 2607:f8b0:400c:c0c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.215.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200402924 (191M) [application/octet-stream]
Saving to: ‘4466624P_sales.csv’

4466624P_sales.csv  100%[===================>] 191.12M   140MB/s    in 1.4s    

2025-06-08 03:30:02 (140 MB/s) - ‘4466624P_sales.csv’ saved [200402924/200402924]

,transaction id,item id,discounted ,discounted %,time of purchase
0,fbfd877b-109b-4477-af5c-267847a7251f,,No,0.0,
1,fbfd877b-109b-4477-af5c-267847a7251f,2ED8,No,0.0,"01/01/2000, 13:33:57"
2,fbfd877b-109b-4477-af5c-267847a7251f,6A06,No,0.0,"01/01/200

### Step 1: Import Libraries and Load Dataset

In [90]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [91]:
# Load and Inspect Data
file_path = "4466624P_sales.csv"
sales_data = pd.read_csv(file_path)

In [92]:
# Display first few rows
sales_data.head()

,Unnamed: 0,transaction id,item id,discounted,discounted %,time of purchase
0,0,fbfd877b-109b-4477-af5c-267847a7251f,NaN,No,0.0,NaN
1,1,fbfd877b-109b-4477-af5c-267847a7251f,2ED8,No,0.0,"01/01/2000, 13:33:57"
2,2,fbfd877b-109b-4477-af5c-267847a7251f,6A06,No,0.0,"01/01/2000, 13:33:25"
3,3,fbfd877b-109b-4477-af5c-267847a7251f,28FC,No,0.0,"01/01/2000, 13:33:04"
4,4,fbfd877b-109b-4477-af5c-267847a7251f,8700,No,0.0,"01/01/2000, 13:33:46"


In [93]:
# Print Schema
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2633416 entries, 0 to 2633415
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Unnamed: 0        int64  
 1   transaction id    object 
 2   item id           object 
 3   discounted        object 
 4   discounted %      float64
 5   time of purchase  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 120.5+ MB


In [94]:
# Count of total records
sales_data.shape[0]

2633416

### Step 2: Data Cleaning

#### Step 2.1: Remove unnecessary column for analysis

First column from our dataset is just a row index or sequence number. We can remove this, since we will not use for analysis.

Removing this column will help to find duplicate records in later step.

In [95]:
# Drop the first column - This is just a row index or sequence number, no use for this. It will help to find duplicate records in later step.
if "unnamed: 0" in sales_data.columns:
    sales_data = sales_data.drop(columns=["unnamed: 0"])

#### Step 2.2: Standardize all column name and value

 We want to ensure our dataset is accurate, consistent, and usable.
 To achieve that, we will perform the following
 - standardize the column names and values, such as converting them to lowercase and removing leading/trailing whitespaces.  
 - removing records with missing value or null
 - remove duplicate records

##### Step 2.2.1: Standardize the column names and values

In [87]:
# Convert column names to lowercase and strip whitespace
sales_data.columns = sales_data.columns.str.lower().str.strip()

# Apply lowercase and strip whitespace for string columns
for col in sales_data.select_dtypes(include='object').columns:
    sales_data[col] = sales_data[col].str.lower().str.strip()

In [88]:
# Check unique values in 'discounted'
print(sales_data['discounted'].unique())

['no' 'yes']


The value for `discounted` column only contains `no` and `yes`, thus no further processing is required

In [67]:
# Check missing values
sales_data.isnull().sum()

,0
transaction id,0
item id,131670
discounted,0
discounted %,0
time of purchase,395012


In [68]:
# Drop rows where item id or time of purchase is missing
# 'item id' is crucial for sales analysis
# 'time of purchase' is crucial for time-based trends
sales_data = sales_data.dropna(subset=['item id', 'time of purchase'])

In [69]:
# Confirm if there is still null value
sales_data.isnull().sum()

,0
transaction id,0
item id,0
discounted,0
discounted %,0
time of purchase,0


In [74]:
# Check total number of duplicate records
duplicate_count = sales_data.duplicated().sum()
print(f"Number of duplicated records: {duplicate_count}")

Number of duplicated records: 1582


In [78]:
duplicate_rows = sales_data[sales_data.duplicated(keep=False)]
display(duplicate_rows)

,transaction id,item id,discounted,discounted %,time of purchase
2573,8b879518-9c2f-447f-b62b-33eeea78c08b,8700,No,0.0,"03/01/2000, 09:02:07"
2575,8b879518-9c2f-447f-b62b-33eeea78c08b,8700,No,0.0,"03/01/2000, 09:02:07"
3046,d2081634-90c8-4287-b5d4-97e2abafb2ae,316D,No,0.0,"03/01/2000, 13:19:04"
3047,d2081634-90c8-4287-b5d4-97e2abafb2ae,316D,No,0.0,"03/01/2000, 13:19:04"
3050,8cd5a5cb-b89e-410c-8592-28f81d707236,D4D3,No,0.0,"03/01/2000, 18:09:38"
...,...,...,...,...,...
2625719,d89e8cf1-b200-4be8-8a8b-11770996e1e0,82FC,No,0.0,"28/12/2009, 16:36:46"
2628206,9ee36fe8-a83b-4684-ae25-d1e42ff54360,1BC3,No,0.0,"30/12/2009, 13:09:19"
2628210,9ee36fe8-a83b-4684-ae25-d1e42ff54360,1BC3,No,0.0,"30/12/2009, 13:09:19"
2628594,c430bb1b-51e5-431c-91d7-8405b7bfde1c,1BC3,No,0.0,"30/12/2009, 09:01:38"
